In [1]:
import os
import pickle
import pandas as pd
from PIL import Image
from tqdm.notebook import tqdm

import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import transforms
from torch.utils.data import Dataset, DataLoader

from new_rpnet import Recognition_Module, Detection_Module

/data2/home/shishirm/.local/lib/python3.8/site-packages/pandas/core/computation/expressions.py:20: UserWarning: Pandas requires version '2.7.3' or newer of 'numexpr' (version '2.7.1' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED


In [2]:
def save_file(history, path):
    with open(path, 'wb') as file:
        pickle.dump(history, file)

def load_file(path):
    with open(path, 'rb') as file:
        history = pickle.load(file)
    return history

In [3]:
device = 'cuda:0'
torch.cuda.empty_cache()

In [4]:
class CustomDataset(Dataset):
    def __init__(self, csv_file, transform=None):
        self.data_frame = pd.read_csv(csv_file)
        self.transform = transform

    def __len__(self):
        return len(self.data_frame)

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()

        image_path = self.data_frame.iloc[idx, 0]
        left = self.data_frame.iloc[idx, 1:3]
        right = self.data_frame.iloc[idx, 3:5]
        label = [x for x in self.data_frame.iloc[idx, 5:]] # license plate chracters' indices

        image = Image.open(image_path).convert('RGB')
        shape = image.size # width x height, 

        box = [(left[0]+right[0])/(2*shape[0]), (left[1]+right[1])/(2*shape[1]), (right[0]-left[0])/shape[0], (right[1]-left[1])/shape[1]]
        # box = [cx, cy, w, h]

        if self.transform:
            image = self.transform(image)

        return image, torch.tensor(box, dtype=torch.float32), torch.tensor(label, dtype=torch.long)

In [5]:
transform = transforms.Compose([
    transforms.Resize((480, 480)),
    transforms.ToTensor(),
])

In [6]:
train_dataset = CustomDataset('../datasets/train.csv', transform = transform)
valid_dataset = CustomDataset('../datasets/validate.csv', transform = transform)

train_loader = DataLoader(train_dataset, batch_size=100, shuffle=True, num_workers=25, persistent_workers=True, prefetch_factor=10)
validation_loader = DataLoader(valid_dataset, batch_size=100, shuffle=False, num_workers=10, persistent_workers=True, prefetch_factor=10)

In [7]:
convert = torch.tensor([[1, 0, 1, 0], [0, 1, 0, 1], [-0.5, 0, 0.5, 0], [0, -0.5, 0, 0.5]]).to(device)
def compute_iou(pred, box):
    boxn = box.mm(convert).clamp(min=0, max=1)
    predn = pred.mm(convert).clamp(min=0, max=1)

    X_min = torch.max(boxn[:, 0], predn[:, 0])
    Y_min = torch.max(boxn[:, 1], predn[:, 1])
    X_max = torch.min(boxn[:, 2], predn[:, 2])
    Y_max = torch.min(boxn[:, 3], predn[:, 3])

    w = nn.functional.relu(X_max - X_min)
    h = nn.functional.relu(Y_max - Y_min)

    a_inter = w * h
    a1 = box[:, 2] * box[:, 3]
    a2 = pred[:, 2] * pred[:, 3]

    iou = a_inter / (a1 + a2 - a_inter)

    return torch.sum(iou).item()

In [8]:
def train_premodel(model, opt, loss_fn, train_loader, lr, num_epochs):
    history = {'train_loss' : [], 'mean_iou' : []}
    optimizer = opt(model.parameters(), lr=lr)

    if os.path.exists('./saved/D_checkpoint.pth'):
        history = load_file('./saved/D_history.pkl')
        checkpoint = torch.load('./saved/D_checkpoint.pth', map_location=device)
        model.load_state_dict(checkpoint['model_state_dict'])
        optimizer.load_state_dict(checkpoint['optimizer_state_dict'])

    with tqdm(total=num_epochs * len(train_loader)) as train_bar:
        for epoch in range(num_epochs):
            if len(history['train_loss']) > epoch:
                train_bar.set_description(f"Resuming")
                train_bar.update(len(train_loader))
                print(f"Epoch {epoch+1} \tTraining Loss : {history['train_loss'][epoch]} \tMean IoU : {history['mean_iou'][epoch]}")
                continue

            train_bar.set_description(f"Training epoch {epoch+1}")
            model.train()
            total = 0.0
            iou = 0.0
            train_loss = 0.0
            for image, box, _ in train_loader:
                image = image.to(device)
                box = box.to(device)

                pred_box = model(image)
                loss = loss_fn(pred_box, box)

                optimizer.zero_grad()
                loss.backward()
                optimizer.step()

                train_loss += loss.item()
                train_bar.update(1)

                total += box.shape[0]
                iou += compute_iou(pred_box, box)

            print(f"Epoch {epoch+1} \tTraining Loss : {train_loss} \tMean IoU : {iou / total}")
            history['train_loss'].append(train_loss)
            history['mean_iou'].append(iou / total)

            torch.save({
                'model_state_dict': model.state_dict(),
                'optimizer_state_dict': optimizer.state_dict(),
            }, './saved/D_checkpoint.pth')
            save_file(history, './saved/D_history.pkl')

    return history

# premodel = Detection_Module()
# premodel.to(device)
# num_parameters = sum(p.numel() for p in premodel.parameters())
# print("Number of parameters in Detection Module =", num_parameters)
# opt = optim.Adam
# loss = nn.L1Loss().to(device)

# history = train_premodel(premodel, opt, loss, train_loader, 0.001, 6)
# torch.save(premodel.state_dict(), "./saved/detection_module.pth")

In [9]:
model = Recognition_Module(device=device, path='./saved/detection_module.pth')
model.to(device)

num_parameters = sum(p.numel() for p in model.parameters())
print("Number of parameters =", num_parameters)

num_parameters = sum(p.numel() for p in model.parameters() if p.requires_grad)
print("Number of trainable parameters =", num_parameters)

Number of parameters = 29654194
Number of trainable parameters = 29654194


In [10]:
opt = optim.Adam
loss1 = nn.L1Loss().to(device)
loss2 = nn.CrossEntropyLoss().to(device)
sch = optim.lr_scheduler.StepLR

In [11]:
def validate_model(model, validation_loader, loss1, loss2):
    validation_loss = 0.0
    ch_correct = 0
    li_correct = 0
    total = 0
    iou = 0
    model.eval()
    with torch.no_grad():
        for image, box, label in validation_loader:
            image = image.to(device)
            box = box.to(device)
            label = label.to(device).T

            pred_box, pred_label = model(image)
            loss = loss1(pred_box, box)
            for i in range(7):
                loss += loss2(pred_label[i], label[i])

            validation_loss += loss.item()

            predictions = [torch.argmax(curr, dim=1) for curr in pred_label]
            pred_license = torch.stack(predictions, dim=1).to(device)
            ch_equal = (label.T == pred_license)
            ch_correct += torch.sum(ch_equal).item()
            li_equal = torch.all(ch_equal, dim=1)
            li_correct += torch.sum(li_equal).item()
            total += box.shape[0]
            iou += compute_iou(pred_box, box)

    return validation_loss, 100 * ch_correct / (7 * total), 100 * li_correct / total, iou / total


In [12]:
def train_model(model, opt, sch, loss1, loss2, train_loader, validation_loader, lr, num_epochs):
    history = {'train_loss' : [], 'val_loss' : [], 'train_ch_acc' : [], 'train_li_acc' : [], 'val_ch_acc' : [], 'val_li_acc' : [], 'val_iou' : [], 'train_iou' : []}

    optimizer = opt(model.parameters(), lr=lr)
    scheduler = sch(optimizer, step_size=6, gamma=0.1)

    if os.path.exists('./saved/P_checkpoint.pth'):
        history = load_file('./saved/P_history.pkl')
        checkpoint = torch.load('./saved/P_checkpoint.pth', map_location=device)
        model.load_state_dict(checkpoint['model_state_dict'])
        optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
        scheduler.load_state_dict(checkpoint['scheduler_state_dict'])

    with tqdm(total=num_epochs * len(train_loader)) as train_bar:
        for epoch in range(num_epochs):
            if len(history['train_loss']) > epoch:
                train_bar.set_description(f"Resuming")
                train_bar.update(len(train_loader))
                print(f"Epoch {epoch+1} \tTraining Loss : {history['train_loss'][epoch]} \tValidation Loss : {history['val_loss'][epoch]}")
                print(f"Train Ch Accuracy : {history['train_ch_acc'][epoch]} \tTrain Li Accuracy : {history['train_li_acc'][epoch]} \tTrain IoU : {history['train_iou'][epoch]}")
                print(f"Validation Ch Accuracy : {history['val_ch_acc'][epoch]} \tValidation Li Accuracy : {history['val_li_acc'][epoch]}  \tValidation IoU : {history['val_iou'][epoch]}")
                print()
                continue
            
            train_bar.set_description(f"Training epoch {epoch+1}")
            model.train()
            train_loss = 0.0
            ch_correct = 0
            li_correct = 0
            total = 0
            iou = 0
            for image, box, label in train_loader:
                image = image.to(device)
                box = box.to(device)
                label = label.to(device).T

                pred_box, pred_label = model(image)
                loss = loss1(pred_box, box)
                for i in range(7):
                    loss += loss2(pred_label[i], label[i])

                optimizer.zero_grad()
                loss.backward()
                optimizer.step()

                train_loss += loss.item()
                train_bar.update(1)

                predictions = [torch.argmax(curr, dim=1) for curr in pred_label]
                pred_license = torch.stack(predictions, dim=1).to(device)
                ch_equal = (label.T == pred_license)
                ch_correct += torch.sum(ch_equal).item()
                li_equal = torch.all(ch_equal, dim=1)
                li_correct += torch.sum(li_equal).item()
                total += box.shape[0]
                iou += compute_iou(pred_box, box)

            train_bar.set_description(f"Validating epoch {epoch+1}")
            val_loss, val_ch_acc, val_li_acc, val_iou = validate_model(model, validation_loader, loss1, loss2)

            train_ch_acc = 100 * ch_correct / (7 * total)
            train_li_acc = 100 * li_correct / total
            train_iou = iou / total

            print(f"Epoch {epoch+1} \tTraining Loss : {train_loss} \tValidation Loss : {val_loss}")
            print(f"Train Ch Accuracy : {train_ch_acc} \tTrain Li Accuracy : {train_li_acc} \tTrain IoU : {train_iou}")
            print(f"Validation Ch Accuracy : {val_ch_acc} \tValidation Li Accuracy : {val_li_acc} \tValidation IoU : {val_iou}")
            print()

            history['train_loss'].append(train_loss)
            history['val_loss'].append(val_loss)
            history['val_ch_acc'].append(val_ch_acc)
            history['val_li_acc'].append(val_li_acc)
            history['train_ch_acc'].append(train_ch_acc)
            history['train_li_acc'].append(train_li_acc)
            history['train_iou'].append(train_iou)
            history['val_iou'].append(val_iou)

            scheduler.step()

            torch.save({
                'model_state_dict': model.state_dict(),
                'optimizer_state_dict': optimizer.state_dict(),
                'scheduler_state_dict': scheduler.state_dict(),
            }, './saved/P_checkpoint.pth')
            save_file(history, './saved/P_history.pkl')

            if epoch % 5 == 4:
                torch.save(model.state_dict(), "./saved/prediction_module_"+str(epoch+1)+".pth")

    return history

In [13]:
history = train_model(model, opt, sch, loss1, loss2, train_loader, validation_loader, 0.001, 30)

  0%|          | 0/36000 [00:00<?, ?it/s]

Epoch 1 	Training Loss : 4957.378397613764 	Validation Loss : 125.50012376904488
Train Ch Accuracy : 83.47387256109974 	Train Li Accuracy : 57.08226038984308 	Train IoU : 0.772848702651689
Validation Ch Accuracy : 97.50857142857143 	Validation Li Accuracy : 88.43 	Validation IoU : 0.8039472831726074

Epoch 2 	Training Loss : 507.0006891563535 	Validation Loss : 74.32626909762621
Train Ch Accuracy : 98.34364906741716 	Train Li Accuracy : 91.91646457828112 	Train IoU : 0.8306942747929864
Validation Ch Accuracy : 98.55357142857143 	Validation Li Accuracy : 92.755 	Validation IoU : 0.835756926727295

Epoch 3 	Training Loss : 330.99966099858284 	Validation Loss : 59.18651216104627
Train Ch Accuracy : 98.90080585347967 	Train Li Accuracy : 94.33735843396084 	Train IoU : 0.8346701372561577
Validation Ch Accuracy : 98.84571428571428 	Validation Li Accuracy : 94.21 	Validation IoU : 0.844189379119873

Epoch 4 	Training Loss : 256.90761490538716 	Validation Loss : 63.40949787758291
Train Ch Accu